<a href="https://colab.research.google.com/github/gkrry2723/ML_pytorch_study/blob/master/2_CNN_fanshionMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl

     |████████████████████████████████| 133.6MB 63kB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12


In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset,DataLoader,TensorDataset)
import tqdm

from torchvision.datasets import FashionMNIST
from torchvision import transforms

fashion_mnist_train = FashionMNIST("/content/FashionMNIST",train =True,download=True,transform=transforms.ToTensor())
fashion_mnist_test = FashionMNIST("/content/FashionMNIST",train =False,download=True,transform=transforms.ToTensor())

batch_size = 128
train_loader = DataLoader(fashion_mnist_train,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(fashion_mnist_test,batch_size=batch_size,shuffle=True)

Extracting /content/FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to /content/FashionMNIST/FashionMNIST/raw


Extracting /content/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /content/FashionMNIST/FashionMNIST/raw



Extracting /content/FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /content/FashionMNIST/FashionMNIST/raw


Extracting /content/FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /content/FashionMNIST/FashionMNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


**CNN 부분**

In [ ]:
#(N,C,H,W) 형식의 텐서를 (N, C*H*W)로 늘리는 게층
#합성곱 출력을 MLP에 전달할 때 필요하다고 한당
class FlattenLayer(nn.Module):
  def forward(self,x):
    size = x.size()
    return x.view(size[0],-1)


#conv : input, output,5*5 커널
#conv 층: 이미지의 특이량을 얻어냄
conv_net = nn.Sequential(
    nn.Conv2d(1,32,5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Dropout2d(0.25),
    nn.Conv2d(32,64,5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Dropout2d(0.25),
    FlattenLayer()
)

# con, maxpool 땜에 그 크기를 예상하기 힘드므로 더미 데이터에 넣어서 확인하기
test_input = torch.ones(1,1,28,28)
conv_output_size = conv_net(test_input).size()[-1]


# 2층 MLP
mlp = nn.Sequential(
    nn.Linear(conv_output_size, 200),
    nn.ReLU(),
    nn.BatchNorm1d(200),
    nn.Dropout(0.25),
    nn.Linear(200, 10)
)
# 최종 CNN
net = nn.Sequential(
    conv_net,
    mlp
)

In [ ]:
# 평가용 헬퍼 함수
def eval_net(net, data_loader, device="cpu"):
    # Dropout 및 BatchNorm을 무효화
    net.eval()
    ys = []
    ypreds = []
    for x, y in data_loader:
        # to 메서드로 계산을 실행할 디바이스로 전송
        x = x.to(device)
        y = y.to(device)
        # 확률이 가장 큰 클래스를 예측(리스트 2.1 참조)
        # 여기선 forward（추론） 계산이 전부이므로 자동 미분에
        # 필요한 처리는 off로 설정해서 불필요한 계산을 제한다
        with torch.no_grad():
            _, y_pred = net(x).max(1)
        ys.append(y)
        ypreds.append(y_pred)
    
    # 미니 배치 단위의 예측 결과 등을 하나로 묶는다
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    # 예측 정확도 계산
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.item()

# 훈련용 헬퍼 함수
def train_net(net, train_loader, test_loader,
              optimizer_cls=optim.Adam,
              loss_fn=nn.CrossEntropyLoss(),
              n_iter=10, device="cpu"):
    train_losses = []
    train_acc = []
    val_acc = []
    optimizer = optimizer_cls(net.parameters())
    for epoch in range(n_iter):
        running_loss = 0.0
        # 신경망을 훈련 모드로 설정
        net.train()
        n = 0
        n_acc = 0
        # 시간이 많이 걸리므로 tqdm을 사용해서 진행바를 표시
        for i, (xx, yy) in tqdm.tqdm(enumerate(train_loader),
            total=len(train_loader)):
            xx = xx.to(device)
            yy = yy.to(device)
            h = net(xx)
            loss = loss_fn(h, yy)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            n += len(xx)
            _, y_pred = h.max(1)
            n_acc += (yy == y_pred).float().sum().item()
        train_losses.append(running_loss / i)
        # 훈련 데이터의 예측 정확도
        train_acc.append(n_acc / n)

        # 검증 데이터의 예측 정확도
        val_acc.append(eval_net(net, test_loader, device))
        # epoch의 결과 표시
        print(epoch, train_losses[-1], train_acc[-1],
            val_acc[-1], flush=True)

In [ ]:
# 신경망의 모든 파라미터를 GPU로 전송
net.to("cuda:0")

# 훈련 실행
train_net(net, train_loader, test_loader, n_iter=20, 
device="cuda:0")


100%|██████████| 469/469 [00:06<00:00, 70.57it/s]


0 0.451356124610473 0.8365666666666667 0.8807999491691589


100%|██████████| 469/469 [00:06<00:00, 73.26it/s]


1 0.31850170716643333 0.8837333333333334 0.8888999819755554


100%|██████████| 469/469 [00:06<00:00, 72.71it/s]


...

17 0.16139314488436168 0.9396833333333333 0.9186999797821045


100%|██████████| 469/469 [00:06<00:00, 72.07it/s]


18 0.1584511808016234 0.9400833333333334 0.9187999963760376


100%|██████████| 469/469 [00:06<00:00, 72.21it/s]


19 0.1573811335505074 0.9408666666666666 0.9199999570846558
